# Resample the existing Raw and Epochs data for CLBP and save as Pickle

In [1]:
import sys
import os
src_path = os.path.dirname(os.path.abspath("../../src"))
sys.path.append(src_path)
from src.preprocessing.classes import Subject
from src.utils.config import Config
import src.configs.config as configs
import src.preprocessing.utils as pre_utils 
import ipyparallel as ipp

# Start a client and view
client = ipp.Client(profile="preprocessing")
dview = client[:]

# Ensure all engines import necessary modules and append the src path
dview.execute(f"import sys; sys.path.append('{src_path}')")
dview.execute('import os')
dview.execute('import pickle')
dview.execute('import mne')
dview.execute('from src.preprocessing.classes import Subject')
dview.execute('from src.utils.config import Config')
dview.execute('import src.configs.config as configs')
dview.execute('import src.preprocessing.utils as pre_utils')

# Load in subject IDs and paths from CFGLog
config = Config.from_json(configs.CFGLog)

CLBP_CP_subject_ids = config.data.chronic_low_back_pain.subject_ids.CP
CLBP_HC_subject_ids = config.data.chronic_low_back_pain.subject_ids.HC

# CLBP already processed
clbp_processed_path = config.data.chronic_low_back_pain.processed_path

# Define preprocessing parameters
# Combine all subject IDs into dict separated by group
sub_ids = {
    "chronic_low_back_pain": CLBP_CP_subject_ids + CLBP_HC_subject_ids
}

print(f"Total subjects: {len(sub_ids['chronic_low_back_pain'])}")
print(sub_ids)

# Push necessary configuration data to all engines
dview.push({
    'clbp_processed_path': clbp_processed_path,
    'sub_ids': sub_ids
})

def preprocess_subject(sub_id, group, clbp_processed_path):
    subject = Subject(sub_id, group)
    subject.preprocessed_data_path = clbp_processed_path
    
    # Load in Raw data
    subject.load_raw()
    
    # Load in Epochs data
    subject.load_epochs()
    
    # Resample both
    subject.raw.resample(pre_utils.RESAMPLE_FREQ)
    subject.epochs.resample(pre_utils.RESAMPLE_FREQ)
    
    # Save as pkl   
    subject.save(subject.raw, clbp_processed_path, "preprocessed_raw")
    for as_mat in [False, True]:
        subject.save(subject.epochs, clbp_processed_path, "epochs", as_mat=as_mat)
    return f"Processed {sub_id} in group {group}"

# Push the preprocess_subject function to all engines
dview.push({'preprocess_subject': preprocess_subject})

# Create a list of tasks
tasks = [(sub_id, group, clbp_processed_path) for group in sub_ids for sub_id in sub_ids[group]]

# Execute tasks in parallel
results = dview.map_sync(lambda args: preprocess_subject(*args), tasks)

# Print results
for result in results:
    print(result)


imported src module
imported preprocessing module
imported configs module
imported utils module
Total subjects: 43
{'chronic_low_back_pain': ['018', '022', '024', '031', '032', '034', '036', '039', '040', '045', '046', '052', '020', '021', '023', '029', '037', '041', '042', '044', '048', '049', '050', '056', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C2.', 'C24', 'C25', 'C26', 'C27', 'C3.', 'C6.', 'C7.', 'C9.']}


CompositeError: one or more exceptions raised in: <lambda>
[0:apply]FileNotFoundError: [Errno 2] No such file or directory: '/home/George Kenefati/Chronic Low Back Pain Study/Data/Raw'
[1:apply]FileNotFoundError: [Errno 2] No such file or directory: '/home/George Kenefati/Chronic Low Back Pain Study/Data/Raw'
[2:apply]FileNotFoundError: [Errno 2] No such file or directory: '/home/George Kenefati/Chronic Low Back Pain Study/Data/Raw'
[3:apply]FileNotFoundError: [Errno 2] No such file or directory: '/home/George Kenefati/Chronic Low Back Pain Study/Data/Raw'
.... 3 more exceptions ...

In [2]:
import sys
import os
src_path = os.path.dirname(os.path.abspath("../../src"))
sys.path.append(src_path)
from src.preprocessing.classes import Subject
from src.utils.config import Config
import src.configs.config as configs
import src.preprocessing.utils as pre_utils 

# Load in subject IDs and paths from CFGLog
config = Config.from_json(configs.CFGLog)

CLBP_CP_subject_ids = config.data.chronic_low_back_pain.subject_ids.CP
CLBP_HC_subject_ids = config.data.chronic_low_back_pain.subject_ids.HC

# CLBP already processed
clbp_processed_path = config.data.chronic_low_back_pain.processed_path

# Define preprocessing parameters
# Combine all subject IDs into dict separated by group
sub_ids = {
    "chronic_low_back_pain": CLBP_CP_subject_ids + CLBP_HC_subject_ids
}

print(f"Total subjects: {len(sub_ids['chronic_low_back_pain'])}")
print(sub_ids)

# Create Subject object for each subject
for group in sub_ids:
    for sub_id in sub_ids[group]:
        subject = Subject(sub_id, group)
        subject.preprocessed_data_path = clbp_processed_path
        
        # Load in Raw data
        subject.load_raw()
        
        # Load in Epochs data
        subject.load_epochs()
        
        # Resample both
        subject.raw.resample(pre_utils.RESAMPLE_FREQ)
        subject.epochs.resample(pre_utils.RESAMPLE_FREQ)
        
        # Save as pkl  
        if not subject.pkl_exists("preprocessed_raw"):
            subject.save(subject.raw, "preprocessed_raw")
        if not subject.pkl_exists("epochs"):
            for as_mat in [False, True]:
                subject.save(subject.epochs, "epochs")
        

Total subjects: 43
{'chronic_low_back_pain': ['018', '022', '024', '031', '032', '034', '036', '039', '040', '045', '046', '052', '020', '021', '023', '029', '037', '041', '042', '044', '048', '049', '050', '056', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C2.', 'C24', 'C25', 'C26', 'C27', 'C3.', 'C6.', 'C7.', 'C9.']}
Extracting EDF parameters from /home/wanglab/Documents/George Kenefati/Chronic Low Back Pain Study/Data/Raw/018.EEGDATA/Acquisition 018.Part2 Data.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 2959599  =      0.000 ...  2959.599 secs...
Loaded raw for subject 018
Reading /home/wanglab/Documents/George Kenefati/Chronic Low Back Pain Study/Data/Processed Data/018_preprocessed-epo.fif ...
    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
73 matching events found
No baseline correction applied
0 project

/home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/src/preprocessing/classes.py:63: RuntimeWarning: Number of records from the header does not match the file size (perhaps the recording was not stopped before exiting). Inferring from the file size.
  self.raw = mne.io.read_raw_edf(self.raw_file_path, preload=True)


Loaded raw for subject 022
Reading /home/wanglab/Documents/George Kenefati/Chronic Low Back Pain Study/Data/Processed Data/022_preprocessed-epo.fif ...
    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
56 matching events found
No baseline correction applied
0 projection items activated
Trigger channel Trigger has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
Trigger channel Trigger has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
Extracting EDF parameters from /home/wanglab/Documents/George Kenefati/Chronic Low Back Pain Study/Data/Raw/024.EEGDATA/024 eeg data.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1921199  =      0.000 ...  1921.199 secs...
Loaded raw for subject 024
Reading /home/wanglab/Documents/George Kenefati/Chronic L

/tmp/ipykernel_43892/1462337590.py:41: RuntimeWarning: Resampling of the stim channels caused event information to become unreliable. Consider finding events on the original data and passing the event matrix as a parameter.
  subject.raw.resample(pre_utils.RESAMPLE_FREQ)


Extracting EDF parameters from /home/wanglab/Documents/George Kenefati/Chronic Low Back Pain Study/Data/Raw/034.EEG DATA.3.4.2021/Acquisition 034 Data.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1713199  =      0.000 ...  1713.199 secs...
Loaded raw for subject 034
Reading /home/wanglab/Documents/George Kenefati/Chronic Low Back Pain Study/Data/Processed Data/034_preprocessed-epo.fif ...
    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
60 matching events found
No baseline correction applied
0 projection items activated
Trigger channel Trigger has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
232 events found on stim channel Trigger
Event IDs: [57342 61439]
Trigger channel Trigger has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)


...

/tmp/ipykernel_43892/1462337590.py:41: RuntimeWarning: Resampling of the stim channels caused event information to become unreliable. Consider finding events on the original data and passing the event matrix as a parameter.
  subject.raw.resample(pre_utils.RESAMPLE_FREQ)


Extracting EDF parameters from /home/wanglab/Documents/George Kenefati/Chronic Low Back Pain Study/Data/Raw/C25.EEG11.07.22/Acquisition 44 Data.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1805599  =      0.000 ...  1805.599 secs...
Loaded raw for subject C25
Reading /home/wanglab/Documents/George Kenefati/Chronic Low Back Pain Study/Data/Processed Data/C25_preprocessed-epo.fif ...
    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
56 matching events found
No baseline correction applied
0 projection items activated
Trigger channel Trigger has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
275 events found on stim channel Trigger
Event IDs: [57342 61439]
Trigger channel Trigger has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
273 eve

/tmp/ipykernel_43892/1462337590.py:41: RuntimeWarning: Resampling of the stim channels caused event information to become unreliable. Consider finding events on the original data and passing the event matrix as a parameter.
  subject.raw.resample(pre_utils.RESAMPLE_FREQ)


Extracting EDF parameters from /home/wanglab/Documents/George Kenefati/Chronic Low Back Pain Study/Data/Raw/C26.EEG3.23.23/Acquisition 45 Data.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 2445599  =      0.000 ...  2445.599 secs...
Loaded raw for subject C26
Reading /home/wanglab/Documents/George Kenefati/Chronic Low Back Pain Study/Data/Processed Data/C26_preprocessed-epo.fif ...
    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
118 matching events found
No baseline correction applied
0 projection items activated
Trigger channel Trigger has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
564 events found on stim channel Trigger
Event IDs: [57343 61439 65535]
Trigger channel Trigger has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
5

/tmp/ipykernel_43892/1462337590.py:41: RuntimeWarning: Resampling of the stim channels caused event information to become unreliable. Consider finding events on the original data and passing the event matrix as a parameter.
  subject.raw.resample(pre_utils.RESAMPLE_FREQ)
